# references
1. https://www.kaggle.com/columbia2131/device-eda-interpolate-by-removing-device-en-ja

# Load Libraries

In [5]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
notebookName = 'Baseline2'
PATH = Path(f"./models/{notebookName}")

In [8]:
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0

In [9]:
num_files

0

# Set Hyper Parameters

In [10]:
RANDOM_STATE = 42
lr = 0.001
batch_size = 4
EPOCH_NUM = 1000


usecols = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM',
             'xSatVelMps', 'ySatVelMps', 'zSatVelMps', 
             'xSatPosM', 'ySatPosM', 'zSatPosM',
            'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec',
         'DriftXRadPerSec' , 'DriftYRadPerSec', 'DriftZRadPerSec',
         'UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT',
         'UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2']
features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM',
            'dlatDeg', 'dlngDeg',
             'xSatVelMps', 'ySatVelMps', 'zSatVelMps', 
             'xSatPosM', 'ySatPosM', 'zSatPosM',
            'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec',
         'DriftXRadPerSec' , 'DriftYRadPerSec', 'DriftZRadPerSec',
         'UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT',
         'UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2']
labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM', 'speedMps', 'courseDegree']

In [11]:
set(features) - set(usecols)

{'dlatDeg', 'dlngDeg'}

In [12]:
torch.manual_seed(RANDOM_STATE)

experience_name = f"{num_files}"
checkpoint_name = "check_point"
model_name = str("model - " + experience_name)
param_name = str("param - " + experience_name)
result_name = str("result - " + experience_name)

dummy_path = str(PATH / f"{num_files}")
checkpoint_path = str(PATH / f"{checkpoint_name}.pth")
model_path = str(PATH / f"{model_name}.pth")
param_path = str(PATH / f"{param_name}.pth")
result_path = str(PATH / f"{result_name}.csv")
model_path, param_path, result_path

('models\\Baseline2\\model - 0.pth',
 'models\\Baseline2\\param - 0.pth',
 'models\\Baseline2\\result - 0.csv')

# Useful Functions

In [13]:
def calc_haversine(lat1, lon1, lat2, lon2):
    dlat = (lat2 - lat1) % 360
    dlon = (lon2 - lon1) % 360 
    
    dlat, dlon = map(np.radians, [dlat, dlon])
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(a ** 0.5)
        
    dist = 6_367_000 * c
    return dist

In [14]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df, meter_score , score

In [15]:
def check_score_np(predict:torch.Tensor, target:torch.Tensor):
    m = []
    predict = predict.detach().numpy()
    target = target.detach().numpy()
    for i in range(predict.shape[0]):
        temp = calc_haversine(predict[i,0], predict[i,1], target[i,0], target[i,1])
        m.append(temp)
    
    m = np.array(m)
    score = (np.percentile(m, 50) + np.percentile(m, 95))/2
    
    return score

# Load Datasets

In [16]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")
df_train = pd.read_pickle(str(data_dir / "gsdc_train.pkl.gzip"))

In [17]:
# check score
df_train, default_loss, default_meas = check_score(df_train)

error meter: 4.261468362435763
score: 5.812326742270749


# Feature Engineering
## Simple view, what is in data frame.

In [18]:
print(df_train.shape)
df_train.head()

(131342, 111)


,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,utcTimeMillis_x,elapsedRealtimeNanos_x,UncalGyroXRadPerSec,...,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM,meter
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423575,-122.094091,-34.06,2020-05-14-US-MTV-1_Pixel4,1.589494e+12,1.965654e+13,-0.000502,...,156.040,3559.757,-468.084,0.001,2.379498e+07,11.992,1134.758,10.866,16.647,4.345847
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423578,-122.094101,-33.29,2020-05-14-US-MTV-1_Pixel4,1.589494e+12,1.965754e+13,-0.003537,...,411.162,-3013.649,65954.431,-0.002,2.592442e+07,3.897,-222.675,7.111,5.174,3.324526
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423573,-122.094111,-30.99,2020-05-14-US-MTV-1_Pixel4,1.589494e+12,1.965854e+13,0.002028,...,-2776.605,-434.445,1223.261,0.000,2.131445e+07,1.499,0.000,4.777,3.371,2.279173
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423583,-122.094121,-32.83,2020-05-14-US-MTV-1_Pixel4,1.589494e+12,1.965954e+13,-0.015753,...,411.755,-3013.528,65954.426,-0.002,2.592544e+07,3.598,-220.611,7.113,5.183,1.381411
4,2020-05-14-US-MTV-1,Pixel4XLModded,1273529466449,37.423574,-122.094137,-33.20,2020-05-14-US-MTV-1_Pixel4XLModded,1.589494e+12,1.734189e+13,0.875268,...,-81.334,-3057.885,-3909.257,0.001,2.318871e+07,10.493,0.000,7.313,5.311,0.794351


In [19]:
for c in df_train.columns:
    print(c)
    print(df_train[c].describe())
    print()

collectionName
count                  131342
unique                     29
top       2021-01-04-US-RWC-1
freq                     8293
Name: collectionName, dtype: object

phoneName
count     131342
unique         7
top       Pixel4
freq       48153
Name: phoneName, dtype: object

millisSinceGpsEpoch
count    1.313420e+05
mean     1.287913e+12
std      1.162110e+10
min      1.273529e+12
25%      1.275424e+12
50%      1.283279e+12
75%      1.302558e+12
max      1.303771e+12
Name: millisSinceGpsEpoch, dtype: float64

latDeg
count    131342.000000
mean         37.435206
std           0.082712
min          37.321683
25%          37.371575
50%          37.424360
75%          37.469109
max          37.690836
Name: latDeg, dtype: float64

lngDeg
count    131342.000000
mean       -122.154925
std           0.145974
min        -122.472214
25%        -122.275109
50%        -122.118311
75%        -122.069091
max        -121.881855
Name: lngDeg, dtype: float64

heightAboveWgs84EllipsoidM
count    1

Name: TimeOffsetNanos, dtype: float64

State
count    131342.000000
mean      37819.338597
std       27131.543312
min           4.000000
25%       16399.000000
50%       23567.000000
75%       49359.000000
max       85034.000000
Name: State, dtype: float64

ReceivedSvTimeNanos
count    1.313420e+05
mean     3.121059e+14
std      1.732955e+14
min      1.328100e+04
25%      1.667148e+14
50%      4.112899e+14
75%      4.270754e+14
max      5.184634e+14
Name: ReceivedSvTimeNanos, dtype: float64

ReceivedSvTimeUncertaintyNanos
count    1.313420e+05
mean     5.091290e+07
std      2.198207e+08
min      2.000000e+00
25%      7.000000e+00
50%      1.200000e+01
75%      2.100000e+01
max      1.000000e+09
Name: ReceivedSvTimeUncertaintyNanos, dtype: float64

Cn0DbHz_x
count    131342.000000
mean         31.992947
std           7.658805
min           7.000000
25%          27.300000
50%          32.400000
75%          37.100000
max          56.600000
Name: Cn0DbHz_x, dtype: float64

PseudorangeRate

max              6.041912e+14            8.309815e+13

yawDeg
count    36589.000000
mean       141.763891
std         89.888553
min          0.000000
25%         73.000000
50%        127.000000
75%        180.000000
max        359.000000
Name: yawDeg, dtype: float64

rollDeg
count    36572.000000
mean        44.125834
std        152.973663
min       -180.000000
25%       -174.000000
50%        146.000000
75%        164.000000
max        179.000000
Name: rollDeg, dtype: float64

pitchDeg
count    36572.000000
mean       -61.415017
std         34.612914
min        -89.000000
25%        -84.000000
50%        -81.000000
75%        -45.000000
max         21.000000
Name: pitchDeg, dtype: float64

timeSinceFirstFixSeconds
count    131342.000000
mean       1213.986316
std         609.145524
min          47.430000
25%         718.430000
50%        1174.430000
75%        1659.440000
max        3216.450000
Name: timeSinceFirstFixSeconds, dtype: float64

hDop
count    131342.000000
mean         10

count    1.313420e+05
mean     2.361523e+07
std      3.328288e+06
min      1.907225e+07
25%      2.180702e+07
50%      2.337290e+07
75%      2.458459e+07
max      4.416339e+07
Name: rawPrM, dtype: float64

rawPrUncM
count    131342.000000
mean          4.881786
std           4.913111
min           0.600000
25%           2.099000
50%           3.298000
75%           5.696000
max         149.896000
Name: rawPrUncM, dtype: float64

isrbM
count    131342.000000
mean       -310.665604
std        1056.395342
min       -6085.269000
25%        -228.997750
50%           0.000000
75%           0.000000
max        9281.743000
Name: isrbM, dtype: float64

ionoDelayM
count    131342.000000
mean          5.713940
std           2.862756
min           0.000000
25%           3.710000
50%           4.895000
75%           7.285000
max          25.876000
Name: ionoDelayM, dtype: float64

tropoDelayM
count    131342.000000
mean          6.304804
std           5.712669
min           0.000000
25%           3

In [20]:
for col in df_train.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
utcTimeMillis_x
elapsedRealtimeNanos_x
UncalGyroXRadPerSec
UncalGyroYRadPerSec
UncalGyroZRadPerSec
DriftXRadPerSec
DriftYRadPerSec
DriftZRadPerSec
utcTimeMillis_y
elapsedRealtimeNanos_y
UncalMagXMicroT
UncalMagYMicroT
UncalMagZMicroT
BiasXMicroT
BiasYMicroT
BiasZMicroT
utcTimeMillis_x
TimeNanos
LeapSecond
TimeUncertaintyNanos
FullBiasNanos
BiasNanos
BiasUncertaintyNanos
DriftNanosPerSecond
DriftUncertaintyNanosPerSecond
HardwareClockDiscontinuityCount
Svid_x
TimeOffsetNanos
State
ReceivedSvTimeNanos
ReceivedSvTimeUncertaintyNanos
Cn0DbHz_x
PseudorangeRateMetersPerSecond
PseudorangeRateUncertaintyMetersPerSecond
AccumulatedDeltaRangeState
AccumulatedDeltaRangeMeters
AccumulatedDeltaRangeUncertaintyMeters
CarrierFrequencyHz_x
CarrierCycles
CarrierPhase
CarrierPhaseUncertainty
MultipathIndicator
SnrInDb
ConstellationType_x
AgcDb
BasebandCn0DbHz_x
FullInterSignalBiasNanos
FullInterSignalBiasUncertai

In [45]:
def FeatureExtraction(df:pd.DataFrame, use_cols, features) -> pd.DataFrame:
    output = df.copy()
    need_create_features = set(features) - set(use_cols)
    
    for needy in need_create_features:
        if needy in ['dlatDeg', 'dlngDeg']:
            output[needy] = DeltaLatLong(df, needy)
    
    return output
    
def DeltaLatLong(df:pd.DataFrame, col):
    output = df.copy()
    output[col] = 0
    if col == 'dlatDeg':
        target = 'latDeg'
    else:
        target = 'lngDeg'
        
    for phone in output['phone'].unique():
        temp = output[output['phone'] == phone]
        origin = temp.reset_index().iloc[0][target]
        output.loc[output['phone'] == phone,col] = temp[target] - origin
    
    return output[col]

df_train = FeatureExtraction(df_train, usecols, features)
df_train.head()
df_train.columns

Index(['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg',
       'lngDeg', 'heightAboveWgs84EllipsoidM', 'phone', 'utcTimeMillis_x',
       'elapsedRealtimeNanos_x', 'UncalGyroXRadPerSec',
       ...
       'satClkBiasM', 'satClkDriftMps', 'rawPrM', 'rawPrUncM', 'isrbM',
       'ionoDelayM', 'tropoDelayM', 'meter', 'dlatDeg', 'dlngDeg'],
      dtype='object', length=113)

In [25]:
class Scaler():
    def __init__(self, data, cols):
        

MinMaxScaler()

In [29]:

scaler.transform(df_train[features])

ValueError: operands could not be broadcast together with shapes (131342,17) (21,) (131342,17) 

In [16]:
df_train[features].info()

TypeError: 'StandardScaler' object is not subscriptable

In [ ]:
df_train = df_train.fillna(method = 'pad')
df_train = df_train.fillna(0)

In [ ]:
df_train[features].describe()

In [ ]:
def data_slice_by_phone(df:pd.DataFrame, 
                        xCols = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'], 
                        yCols = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM', 'speedMps', 'courseDegree']):
    indexCols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch']
    
    indx = []
    data = []
    true = []
    for i, phone in enumerate(df['phone'].unique()):
        temp = df[df['phone'] == phone]
        indx.append(temp[indexCols])
        data.append(temp[xCols])
        true.append(temp[yCols])
        
    return indx, data, true

def data_merge_by_index(df:pd.DataFrame, index:pd.DataFrame, src:pd.DataFrame, srcCols = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']):
    indexCols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch']
    output = df.copy()
    
    listFrame = []
    for i in range(len(index)):
        idxFrame = index[i]
        srcFrame = src[i]
        temp = pd.concat([idxFrame, srcFrame], axis = 1)
        listFrame.append(temp)
    
    dataFrame = pd.concat(listFrame, axis = 0)
    output = output.merge(dataFrame,
              on=["collectionName", "phoneName", "millisSinceGpsEpoch"])
    
    return output

idxFrame, dataFrame, trueFrame = data_slice_by_phone(df_train, xCols=features, yCols = labels)

# Check!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
slide window 기반은 때려치고, 데이터프레임에서 직접 읽어오는 방식으로 변경할 필요가 있음
collectionName, phoneName기반으로 데이터를 잘라서 시계열 데이터처럼 쓸 예정 우선적으로 필요한 것은 lstm으로 돌려보기(baseline을 그렇게 하자!)

# Modeling

In [ ]:
# build model
class ConvBlock(nn.Module):
    def __init__(self, input_features, output_features, kernel_size = 3):
        super().__init__()
        
        self.convK = nn.Conv1d(input_features, output_features, kernel_size = kernel_size, padding = kernel_size//2)
        self.conv = nn.Conv1d(input_features, output_features, 1)
        self.bn = nn.BatchNorm1d(output_features)
        
    def forward(self, inputs):
        x = self.convK(inputs)
        inputs = self.conv(inputs)
        x = self.bn(x)
        x = F.relu(x)
        outputs = x + inputs
        return outputs
    
class BigConv(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.conv1 = ConvBlock(input_size, 2*input_size)
        self.conv2 = ConvBlock(2*input_size, 4*input_size)
        self.conv3 = ConvBlock(4*input_size, 4*input_size)
        self.conv4 = ConvBlock(4*input_size, 4*input_size)
        self.conv5 = ConvBlock(4*input_size, output_size)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return x
    
    
    
class ConvolutionNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
#         self.conv = nn.Conv1d(input_size, output_size, 1)
        self.conv1 = BigConv(input_size, 64)
        self.conv2 = BigConv(64, 64)
        self.conv3 = BigConv(64, 64)
        self.conv4 = BigConv(64, output_size)
        
#         self.lstm = nn.LSTM(input_size = 512 + output_size, hidden_size = output_size, num_layers = 3, dropout = 0.3)
    
    def forward(self, x):
#         x1 = self.conv(x)
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
#         x = torch.cat([x1, x], axis = 1)
#         x, hidden = self.lstm(x)
        
        return x
    
model = ConvolutionNetwork(len(features), len(labels))
model.to(device)

# loss_func = nn.SmoothL1Loss()
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = lr)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                lr_lambda=lambda epoch: 0.995 ** epoch,
                                last_epoch=-1,
                                verbose=True)

In [ ]:
class CustomDataloader(torch.utils.data.Dataset):
    def __init__(self, data, true = None):
        self.data = data
        self.true = true
        self.len = len(data)
        
    
    def __len__(self):
        return self.len
    
    def __shuffle__(self):
        index = np.array(range(self.len), dtype = 'int')
        index = np.random.permutation(index)
        data = []
        true = []
        for i in range(self.len):
            data.append(self.data[index[i]])
            true.append(self.true[index[i]])
            
        self.data = data
        self.true = true
    
    def __getitem__(self, idx):
        data = self.data[idx]
        if self.true is None:
            true = []
        else:
            true = self.true[idx]
            
        data = torch.Tensor(np.expand_dims(np.array(data), 0)).transpose(2, 1)
        true = torch.Tensor(np.expand_dims(np.array(true), 0)).transpose(2, 1)
        
        return data, true
    

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dataFrame, trueFrame, test_size = 1/10)

loader_train = CustomDataloader(Xtrain, ytrain)
loader_test = CustomDataloader(Xtest, ytest)

In [ ]:
def train(epoch):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    loader_train.__shuffle__()
    
    for data, targets in loader_train:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs.squeeze(dim = 0).transpose(1,0))
        ground.append(targets.squeeze(dim = 0).transpose(1,0))
    scheduler.step()

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = check_score_np(predict.to('cpu'), ground.to('cpu'))
    return loss, meas

In [ ]:
def test():
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets in loader_test:
            data = data.to(device)
            targets = targets.to(device)
            
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            predict.append(outputs.squeeze(dim = 0).transpose(1,0))
            ground.append(targets.squeeze(dim = 0).transpose(1,0))

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = check_score_np(predict.to('cpu'), ground.to('cpu'))
    return loss, meas

In [ ]:
test()

In [ ]:
history = []
train_loss, train_meas = 0,0
test_loss, test_meas = 0,0

check_meas = np.inf
check_loss = np.inf
check_epoch = 0

for epoch in range(EPOCH_NUM+1):
    if epoch != 0:
        train_loss, train_meas = train(epoch)
    test_loss, test_meas = test()
    
    history.append({'epoch':epoch, 'train_loss':train_loss, 'train_meas':train_meas, 'test_loss':test_loss, 'test_meas':test_meas})
    
    if (test_meas < check_meas):
        print("")
        print(f"/***CHECK_POINT***/ ")
        print(f"TRAIN - {train_loss}, {train_meas}")
        print(f"TEST - {test_loss}, {test_meas}")
        print("")
        check_meas = test_meas
        check_loss = test_loss
        check_epoch = epoch
        torch.save(model.state_dict(), checkpoint_path)
    
    print(f"/*** EPOCH : {epoch}/{EPOCH_NUM} ***/")
    print(f"TRAIN - {train_loss}, {train_meas}")
    print(f"TEST - {test_loss}, {test_meas}")
    print("")
    
df_history = pd.DataFrame(history)

In [ ]:
df_history = pd.DataFrame(history)

fig, axes = plt.subplots(2,1,figsize = (8,12))

axes[0].plot(df_history['epoch'], df_history['train_loss'])
axes[0].plot(df_history['epoch'], df_history['test_loss'])
axes[0].axvline(x = check_epoch, ymin = 0, ymax = df_history['test_loss'].max(), color = 'r')
axes[0].axhline(y = default_loss, xmin = 0, xmax = df_history['epoch'].max(), color = 'k')
axes[0].set_ylim(0, 20000)

axes[1].plot(df_history['epoch'], df_history['train_meas'])
axes[1].plot(df_history['epoch'], df_history['test_meas'])
axes[1].axvline(x = check_epoch, ymin = 0, ymax = df_history['test_meas'].max(), color = 'r')
axes[1].axhline(y = default_meas, xmin = 0, xmax = df_history['epoch'].max(), color = 'k')

In [ ]:
del X_train, X_test, y_train, y_test

# Submission

In [ ]:
df_test = pd.read_pickle(str(data_dir / "gsdc_test.pkl.gzip"))

print(df_test.shape)
df_test.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
torch.save(model.state_dict(), model_path)

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)

In [ ]:
model = SimpleNetwork(X.shape[1] * X.shape[2], y.shape[1])
model = model.to(device)
model.load_state_dict(torch.load(model_path))

In [ ]:
X, _ = extract_features(df_test, train = False)

X = torch.Tensor(X)

loader_test = DataLoader(X, batch_size = batch_size, shuffle = False)

In [ ]:
model.eval()  # 신경망을 추론 모드로 전환

# 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
predict = []
with torch.no_grad():  # 추론 과정에는 미분이 필요없음
    for data in loader_test:
        data = data.to(device)

        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        predict.append(outputs)
predict = torch.cat(predict, axis = 0).to('cpu')

In [ ]:
predict.shape

In [ ]:
print(submission.shape)
submission.columns

In [ ]:
submission['latDeg'] = predict[:,1]
submission['lngDeg'] = predict[:,2]

In [ ]:
submission.to_csv(f"./models/{notebookName}/{num_files} - result.csv", index = False)
pd.DataFrame([]).to_csv(dummy_path)